# Генеративне мреже

Рекурентне неуронске мреже (RNNs) и њихове варијанте са "гейтованим" ћелијама, као што су Long Short Term Memory ћелије (LSTMs) и Gated Recurrent Units (GRUs), пружају механизам за моделирање језика, тј. могу да науче редослед речи и дају предвиђања за следећу реч у низу. Ово нам омогућава да користимо RNNs за **генеративне задатке**, као што су обично генерисање текста, машински превод, па чак и генерисање описа слика.

У RNN архитектури коју смо разматрали у претходној јединици, свака RNN јединица је производила следеће скривено стање као излаз. Међутим, можемо додати још један излаз свакој рекурентној јединици, што би нам омогућило да добијемо **низ** (који је једнаке дужине као оригинални низ). Штавише, можемо користити RNN јединице које не прихватају улаз на сваком кораку, већ само узимају неки почетни вектор стања и затим производе низ излаза.

У овом нотебуку, фокусираћемо се на једноставне генеративне моделе који нам помажу да генеришемо текст. Ради једноставности, направимо **мрежу на нивоу карактера**, која генерише текст слово по слово. Током тренинга, потребно је узети неки корпус текста и поделити га на низове слова.


In [1]:
import torch
import torchtext
import numpy as np
from torchnlp import *
train_dataset,test_dataset,classes,vocab = load_dataset()

Loading dataset...
Building vocab...


## Изградња вокабулара карактера

Да бисмо изградили генеративну мрежу на нивоу карактера, потребно је да текст поделимо на појединачне карактере уместо на речи. Ово се може урадити дефинисањем другачијег токенизатора:


In [2]:
def char_tokenizer(words):
    return list(words) #[word for word in words]

counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(char_tokenizer(line))
vocab = torchtext.vocab.vocab(counter)

vocab_size = len(vocab)
print(f"Vocabulary size = {vocab_size}")
print(f"Encoding of 'a' is {vocab.get_stoi()['a']}")
print(f"Character with code 13 is {vocab.get_itos()[13]}")

Vocabulary size = 82
Encoding of 'a' is 1
Character with code 13 is c


Хајде да видимо пример како можемо да кодирамо текст из нашег скупа података:


In [3]:
def enc(x):
    return torch.LongTensor(encode(x,voc=vocab,tokenizer=char_tokenizer))

enc(train_dataset[0][1])

tensor([ 0,  1,  2,  2,  3,  4,  5,  6,  3,  7,  8,  1,  9, 10,  3, 11,  2,  1,
        12,  3,  7,  1, 13, 14,  3, 15, 16,  5, 17,  3,  5, 18,  8,  3,  7,  2,
         1, 13, 14,  3, 19, 20,  8, 21,  5,  8,  9, 10, 22,  3, 20,  8, 21,  5,
         8,  9, 10,  3, 23,  3,  4, 18, 17,  9,  5, 23, 10,  8,  2,  2,  8,  9,
        10, 24,  3,  0,  1,  2,  2,  3,  4,  5,  9,  8,  8,  5, 25, 10,  3, 26,
        12, 27, 16, 26,  2, 27, 16, 28, 29, 30,  1, 16, 26,  3, 17, 31,  3, 21,
         2,  5,  9,  1, 23, 13, 32, 16, 27, 13, 10, 24,  3,  1,  9,  8,  3, 10,
         8,  8, 27, 16, 28,  3, 28,  9,  8,  8, 16,  3,  1, 28,  1, 27, 16,  6])

## Тренирање генеративне RNN

Начин на који ћемо тренирати RNN за генерисање текста је следећи. На сваком кораку, узимамо секвенцу карактера дужине `nchars` и тражимо од мреже да генерише следећи излазни карактер за сваки улазни карактер:

![Слика која приказује пример RNN генерисања речи 'HELLO'.](../../../../../translated_images/sr/rnn-generate.56c54afb52f9781d.webp)

У зависности од конкретног сценарија, можда ћемо желети да укључимо неке посебне карактере, као што је *крај секвенце* `<eos>`. У нашем случају, желимо само да обучимо мрежу за бесконачно генерисање текста, па ћемо фиксирати величину сваке секвенце да буде једнака `nchars` токенима. Сходно томе, сваки пример за тренирање ће се састојати од `nchars` улаза и `nchars` излаза (што је улазна секвенца померена за један симбол улево). Минибатч ће се састојати од неколико таквих секвенци.

Начин на који ћемо генерисати минибатчеве је да узмемо сваки текст вести дужине `l` и генеришемо све могуће комбинације улаз-излаз из њега (биће `l-nchars` таквих комбинација). Оне ће чинити један минибатч, а величина минибатчева ће бити различита на сваком кораку тренирања.


In [4]:
nchars = 100

def get_batch(s,nchars=nchars):
    ins = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    outs = torch.zeros(len(s)-nchars,nchars,dtype=torch.long,device=device)
    for i in range(len(s)-nchars):
        ins[i] = enc(s[i:i+nchars])
        outs[i] = enc(s[i+1:i+nchars+1])
    return ins,outs

get_batch(train_dataset[0][1])

(tensor([[ 0,  1,  2,  ..., 28, 29, 30],
         [ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         ...,
         [20,  8, 21,  ...,  1, 28,  1],
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16]]),
 tensor([[ 1,  2,  2,  ..., 29, 30,  1],
         [ 2,  2,  3,  ..., 30,  1, 16],
         [ 2,  3,  4,  ...,  1, 16, 26],
         ...,
         [ 8, 21,  5,  ..., 28,  1, 27],
         [21,  5,  8,  ...,  1, 27, 16],
         [ 5,  8,  9,  ..., 27, 16,  6]]))

Сада ћемо дефинисати генераторску мрежу. Она може бити заснована на било којој рекурентној ћелији коју смо разматрали у претходној јединици (једноставна, LSTM или GRU). У нашем примеру користићемо LSTM.

Пошто мрежа узима карактере као улаз, а величина вокабулара је прилично мала, не треба нам слој за уграђивање; улаз кодирани у један бит може директно ићи у LSTM ћелију. Међутим, пошто прослеђујемо бројеве карактера као улаз, потребно је да их кодирујемо у један бит пре него што их проследимо LSTM-у. Ово се ради позивом функције `one_hot` током `forward` пролаза. Енкодер излаза биће линеарни слој који ће скривено стање претворити у излаз кодирани у један бит.


In [5]:
class LSTMGenerator(torch.nn.Module):
    def __init__(self, vocab_size, hidden_dim):
        super().__init__()
        self.rnn = torch.nn.LSTM(vocab_size,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, s=None):
        x = torch.nn.functional.one_hot(x,vocab_size).to(torch.float32)
        x,s = self.rnn(x,s)
        return self.fc(x),s

Током тренинга, желимо да будемо у могућности да узоркујемо генерисани текст. Да бисмо то постигли, дефинисаћемо функцију `generate` која ће производити излазни низ дужине `size`, почевши од почетног низа `start`.

Начин на који ово функционише је следећи. Прво, проследићемо цео почетни низ кроз мрежу и добити излазно стање `s` и следећи предвиђени карактер `out`. Пошто је `out` једнократно кодирано, узимамо `argmax` да бисмо добили индекс карактера `nc` у вокабулару, и користимо `itos` да бисмо утврдили стварни карактер и додали га резултујућој листи карактера `chars`. Овај процес генерисања једног карактера се понавља `size` пута како би се генерисао потребан број карактера.


In [8]:
def generate(net,size=100,start='today '):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            nc = torch.argmax(out[0][-1])
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)

Хајде да започнемо са тренингом! Петља за тренинг је скоро иста као у свим нашим претходним примерима, али уместо тачности, штампамо узорковани генерисани текст сваких 1000 епоха.

Посебну пажњу треба обратити на начин на који рачунамо губитак. Потребно је израчунати губитак на основу једнохот-кодованог излаза `out` и очекиваног текста `text_out`, који је листа индекса карактера. Срећом, функција `cross_entropy` очекује ненормализовани излаз мреже као први аргумент, а број класе као други, што је управо оно што имамо. Такође, она аутоматски врши просек преко величине мини-бача.

Ограничавамо тренинг на `samples_to_train` узорака, како не бисмо предуго чекали. Охрабрујемо вас да експериментишете и пробате дужи тренинг, могуће током неколико епоха (у том случају би требало да направите још једну петљу око овог кода).


In [9]:
net = LSTMGenerator(vocab_size,64).to(device)

samples_to_train = 10000
optimizer = torch.optim.Adam(net.parameters(),0.01)
loss_fn = torch.nn.CrossEntropyLoss()
net.train()
for i,x in enumerate(train_dataset):
    # x[0] is class label, x[1] is text
    if len(x[1])-nchars<10:
        continue
    samples_to_train-=1
    if not samples_to_train: break
    text_in, text_out = get_batch(x[1])
    optimizer.zero_grad()
    out,s = net(text_in)
    loss = torch.nn.functional.cross_entropy(out.view(-1,vocab_size),text_out.flatten()) #cross_entropy(out,labels)
    loss.backward()
    optimizer.step()
    if i%1000==0:
        print(f"Current loss = {loss.item()}")
        print(generate(net))

Current loss = 4.398899078369141
today sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr sr s
Current loss = 2.161320447921753
today and to the tor to to the tor to to the tor to to the tor to to the tor to to the tor to to the tor t
Current loss = 1.6722588539123535
today and the court to the could to the could to the could to the could to the could to the could to the c
Current loss = 2.423795223236084
today and a second to the conternation of the conternation of the conternation of the conternation of the 
Current loss = 1.702607274055481
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.692358136177063
today and the company to the company to the company to the company to the company to the company to the co
Current loss = 1.9722288846969604
today and the control the control the control the control the control the control the control the control 
Current loss = 1.8

Овај пример већ генерише прилично добар текст, али може се додатно побољшати на неколико начина:

* **Боља генерација минибатчева**. Начин на који смо припремили податке за тренирање био је да генеришемо један минибач из једног узорка. Ово није идеално, јер су минибачеви различитих величина, а неки од њих чак не могу бити генерисани, јер је текст мањи од `nchars`. Такође, мали минибачеви не оптерећују довољно GPU. Паметније би било узети један велики део текста из свих узорака, затим генерисати све парове улаз-излаз, промешати их и генерисати минибачеве једнаке величине.

* **Вишеслојни LSTM**. Има смисла пробати са 2 или 3 слоја LSTM ћелија. Као што смо поменули у претходној јединици, сваки слој LSTM-а извлачи одређене обрасце из текста, а у случају генератора на нивоу карактера можемо очекивати да је нижи ниво LSTM-а одговоран за извлачење слогова, а виши нивои - за речи и комбинације речи. Ово се може једноставно имплементирати прослеђивањем параметра броја слојева конструктору LSTM-а.

* Такође можете експериментисати са **GRU јединицама** и видети које боље раде, као и са **различитим величинама скривених слојева**. Превелики скривени слој може довести до пренапрегнутости (нпр. мрежа ће научити тачан текст), док мања величина можда неће дати добар резултат.


## Генерисање меког текста и температура

У претходној дефиницији функције `generate`, увек смо узимали карактер са највећом вероватноћом као следећи карактер у генерисаном тексту. То је доводило до тога да се текст често "врти" између истих секвенци карактера изнова и изнова, као у овом примеру:
```
today of the second the company and a second the company ...
```

Међутим, ако погледамо расподелу вероватноћа за следећи карактер, може се десити да разлика између неколико највећих вероватноћа није велика, на пример, један карактер може имати вероватноћу 0.2, а други 0.19, итд. На пример, када тражимо следећи карактер у секвенци '*play*', следећи карактер може једнако добро бити размак или **e** (као у речи *player*).

Ово нас доводи до закључка да није увек "праведно" изабрати карактер са већом вероватноћом, јер избор другог највећег може и даље довести до смисленог текста. Паметније је **узорковати** карактере из расподеле вероватноћа коју даје излаз мреже.

Ово узорковање може се обавити помоћу функције `multinomial` која имплементира такозвану **мултиномијалну расподелу**. Функција која имплементира ово **меко** генерисање текста дефинисана је испод:


In [10]:
def generate_soft(net,size=100,start='today ',temperature=1.0):
        chars = list(start)
        out, s = net(enc(chars).view(1,-1).to(device))
        for i in range(size):
            #nc = torch.argmax(out[0][-1])
            out_dist = out[0][-1].div(temperature).exp()
            nc = torch.multinomial(out_dist,1)[0]
            chars.append(vocab.get_itos()[nc])
            out, s = net(nc.view(1,-1),s)
        return ''.join(chars)
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"--- Temperature = {i}\n{generate_soft(net,size=300,start='Today ',temperature=i)}\n")

--- Temperature = 0.3
Today and a company and complete an all the land the restrational the as a security and has provers the pay to and a report and the computer in the stand has filities and working the law the stations for a company and with the company and the final the first company and refight of the state and and workin

--- Temperature = 0.8
Today he oniis its first to Aus bomblaties the marmation a to manan  boogot that pirate assaid a relaid their that goverfin the the Cappets Ecrotional Assonia Cition targets it annight the w scyments Blamity #39;s TVeer Diercheg Reserals fran envyuil that of ster said access what succers of Dour-provelith

--- Temperature = 1.0
Today holy they a 11 will meda a toket subsuaties, engins for Chanos, they's has stainger past to opening orital his thempting new Nattona was al innerforder advan-than #36;s night year his religuled talitatian what the but with Wednesday to Justment will wemen of Mark CCC Camp as Timed Nae wome a leaders

--- Temper

Увели смо још један параметар који се зове **температура**, који се користи да означи колико строго треба да се држимо највеће вероватноће. Ако је температура 1.0, радимо правично мултиномијално узорковање, а када температура иде ка бесконачности - све вероватноће постају једнаке, и насумично бирамо следећи карактер. У примеру испод можемо приметити да текст постаје бесмислен када превише повећамо температуру, и подсећа на "циклично" тешко генерисан текст када се приближи 0.



---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем услуге за превођење помоћу вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако се трудимо да обезбедимо тачност, молимо вас да имате у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на његовом изворном језику треба сматрати ауторитативним извором. За критичне информације препоручује се професионални превод од стране људи. Не преузимамо одговорност за било каква погрешна тумачења или неспоразуме који могу настати услед коришћења овог превода.
